# Aggregating and putting together all format data

Going to put it all together now I think

In [1]:
import pandas as pd 
import json 
import requests 

In [4]:
#api call and cleaning
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
oracle_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "oracle_cards"').index[0]]

response = requests.get(oracle_uri)
raw_oracle_cards = pd.json_normalize(response.json())

oracle_cards = raw_oracle_cards[[#'object', 'id', 
       'oracle_id', 
       #'multiverse_ids', 'mtgo_id',
       #'mtgo_foil_id', 'tcgplayer_id', 'cardmarket_id', 
       'name', 
       #'lang',
       #'released_at', 'uri', 'scryfall_uri', 
       'layout', 
       #'highres_image',
       #'image_status', 'mana_cost', 'cmc', 
       'type_line', 
       #'oracle_text',
       #'colors', 'color_identity', 'keywords', 'games', 'reserved',
       #'foil', 'nonfoil', 'finishes', 'oversized', 'promo', 'reprint',
       #'variation', 'set_id', 'set', 'set_name', 'set_type', 'set_uri',
       #'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       #'prints_search_uri', 'collector_number', 'digital', 
       #'rarity',
       #'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       #'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       #'booster', 'story_spotlight', 'edhrec_rank', 'image_uris.small',
       #'image_uris.normal', 'image_uris.large', 'image_uris.png',
       #'image_uris.art_crop', 'image_uris.border_crop',
       'legalities.standard',
       #'legalities.future', 'legalities.historic',
       #'legalities.gladiator', 
       'legalities.pioneer',
       #'legalities.explorer', 
       'legalities.modern', 
       'legalities.legacy',
       'legalities.pauper', 
       'legalities.vintage', 
       #'legalities.penny',
       #'legalities.commander', 'legalities.brawl',
       #'legalities.historicbrawl', 'legalities.alchemy',
       #'legalities.paupercommander', 'legalities.duel',
       #'legalities.oldschool', 'legalities.premodern', 
       'prices.usd',
       #'prices.usd_foil', 'prices.usd_etched', 'prices.eur',
       #'prices.eur_foil', 
       'prices.tix', 
       #'related_uris.gatherer',
       #'related_uris.tcgplayer_infinite_articles',
       #'related_uris.tcgplayer_infinite_decks', 'related_uris.edhrec',
       #'security_stamp', 'preview.source', 'preview.source_uri',
       #'preview.previewed_at', 'power', 'toughness', 'penny_rank',
       #'arena_id', 'watermark', 'produced_mana', 'all_parts',
       'card_faces', 
       #'frame_effects', 'tcgplayer_etched_id',
       #'promo_types', 'loyalty', 'life_modifier', 'hand_modifier',
       #'attraction_lights', 'color_indicator', 'content_warning'
       ]]
oracle_cards = oracle_cards.rename(columns = {
    'name':'Name',
    'rarity':'Rarity',
    'legalities.pauper':'Pauper_Legal',
    'legalities.standard':'Standard_Legal',
    'legalities.pioneer':'Pioneer_Legal',
    'legalities.modern':'Modern_Legal',
    'legalities.legacy':'Legacy_Legal',
    'legalities.vintage':'Vintage_Legal',
    'prices.usd':'Price_USD',
    'prices.tix':'Price_Tix'
})

oracle_cards = oracle_cards.query("Standard_Legal == 'legal' or Pioneer_Legal == 'legal' or Modern_Legal == 'legal' or Legacy_Legal == 'legal' or Vintage_Legal == 'restricted' or Vintage_Legal == 'legal'")

card_faces = pd.json_normalize(oracle_cards['card_faces'].loc[~oracle_cards['card_faces'].isna()])
front_face = pd.json_normalize(card_faces[0])
back_face = pd.json_normalize(card_faces[1])

faces_index = 0 # we lose the relationship between the indices when we normalize the faces data, I'm not sure how to fix this with pandas but just keeping track of it like this works fine
for card_index in oracle_cards.loc[~oracle_cards['card_faces'].isna()].index:
    if oracle_cards['layout'].loc[card_index] == 'split':
        oracle_cards['Name'].loc[card_index] = front_face['name'].loc[faces_index]+'/'+back_face['name'].loc[faces_index]
    else:
        oracle_cards["Name"].loc[card_index] = front_face['name'].loc[faces_index]
    faces_index += 1


oracle_cards.head()

,oracle_id,Name,layout,type_line,Standard_Legal,Pioneer_Legal,Modern_Legal,Legacy_Legal,Pauper_Legal,Vintage_Legal,Price_USD,Price_Tix,card_faces
0,0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,normal,Artifact,not_legal,not_legal,not_legal,legal,not_legal,legal,21.46,0.51,NaN
1,0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,normal,Enchantment — Aura,not_legal,legal,legal,legal,legal,legal,0.02,0.04,NaN
2,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,normal,Sorcery,not_legal,not_legal,not_legal,legal,not_legal,legal,0.26,0.66,NaN
3,000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,normal,Creature — Bird,not_legal,not_legal,legal,legal,legal,legal,0.21,0.03,NaN
4,000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,normal,Creature — Sponge,not_legal,not_legal,not_legal,legal,not_legal,legal,0.21,0.09,NaN


The following bit of code finds prices for cards that don't have them listed with the oracle cards

In [5]:
bulk = requests.get('https://api.scryfall.com/bulk-data') #I learned that the uri for bulk data changes daily, so requesting the bulk data for the day then getting the uri for updated cards is needed (I had outdated data)
bulk_df = pd.json_normalize(bulk.json(),'data')
default_uri = bulk_df['download_uri'].loc[bulk_df.query('type == "default_cards"').index[0]]

response = requests.get(default_uri)
default_cards = pd.json_normalize(response.json())


no_price_cards =  oracle_cards.loc[oracle_cards['Price_USD'].isna()] #dataframe for keeping track of the priceless cards

for oraclecard_index in no_price_cards.index:#for each card we don't have price for in the oracle card list
    card_printings = default_cards.query('oracle_id == "'+no_price_cards['oracle_id'].loc[oraclecard_index]+'"') #make a temporary dataframe for each printing that occurs in the defaulat card list
    prices_list = []
    for printing_index in card_printings.index: # for each printing of the card
        if card_printings['prices.usd'].loc[printing_index]: #if we can find a nonzero price, 
            prices_list.append(card_printings['prices.usd'].loc[printing_index]) #add it to the list of prices
        if card_printings['prices.usd_foil'].loc[printing_index]: #repeat this logic for foil cards (some cards only have this)
            prices_list.append(card_printings['prices.usd_foil'].loc[printing_index])
    #make the price the lowest found!
    if prices_list != []:
        oracle_cards['Price_USD'].loc[oraclecard_index] = min(prices_list)

In [6]:
oracle_cards.loc[oracle_cards['Price_USD'].isna()] #the only cards we are missing for now are meld cards and for some reason mox sapphire; it only has prices in euros right now

,oracle_id,Name,layout,type_line,Standard_Legal,Pioneer_Legal,Modern_Legal,Legacy_Legal,Pauper_Legal,Vintage_Legal,Price_USD,Price_Tix,card_faces
6254,3989308a-4c0f-493a-b38f-cb4f5019dfe7,"Mishra, Lost to Phyrexia",meld,Legendary Artifact Creature — Phyrexian Artificer,legal,legal,legal,legal,not_legal,legal,None,None,NaN
7333,43230d70-4d60-4ceb-bb45-fbc043280652,"Titania, Gaea Incarnate",meld,Legendary Creature — Elemental Avatar,legal,legal,legal,legal,not_legal,legal,None,None,NaN
8555,4e81596c-9225-43d1-bd35-798212144f2c,Juzám Djinn,normal,Creature — Djinn,not_legal,not_legal,not_legal,legal,not_legal,legal,None,0.53,NaN
11473,6923cf6b-7d3e-4d95-abaf-1df1a04ac7c1,Chittering Host,meld,Creature — Eldrazi Horror,not_legal,legal,legal,legal,legal,legal,None,None,NaN
12848,759406d7-44ae-4260-9ef5-3bb2c92f751a,"Urza, Planeswalker",meld,Legendary Planeswalker — Urza,legal,legal,legal,legal,not_legal,legal,None,None,NaN
19525,b23587e0-9fb7-492e-8bb4-f218e8e4ce6c,"Brisela, Voice of Nightmares",meld,Legendary Creature — Eldrazi Angel,not_legal,legal,legal,legal,not_legal,legal,None,None,NaN
21393,c3327ffe-f47f-4f5b-b929-a7706ef4932d,Kongming's Contraptions,normal,Creature — Human Soldier,not_legal,not_legal,not_legal,legal,not_legal,legal,None,None,NaN
23501,d5ed1233-df87-4b90-8918-13922ec95249,Mox Sapphire,normal,Artifact,not_legal,not_legal,not_legal,banned,not_legal,restricted,None,5.66,NaN
26900,f4905c40-003e-4992-b8d7-3f07ba09c686,"Hanweir, the Writhing Township",meld,Legendary Creature — Eldrazi Ooze,not_legal,legal,legal,legal,not_legal,legal,None,None,NaN


In [7]:
#adding Is_Land values
oracle_cards['Is_Land'] = ''

for i in oracle_cards.index:
    if 'Land' in oracle_cards['type_line'].loc[i]:
        oracle_cards['Is_Land'].loc[i] = True
    else:
        oracle_cards['Is_Land'].loc[i] = False

oracle_cards = oracle_cards.drop(['type_line','layout','card_faces'], axis = 1)

In [8]:
format_names = ['Pauper','Standard','Pioneer','Modern','Legacy','Vintage']
pauper_deck_names = ['Burn','Affinity','Dimir Control','Mono-Blue Faeries','Orzhov Ephemerate','Gruul Ponza','Familiars',
         'Boros Synthesizer','Mono-Red Synthesizer','Walls Combo','Tron','Caw Gates','4 Color Ephemerate','Ephemerate Tron',
         'Bogles','Kiln Fiend','Boros Bully','Izzet Faeries','Goblins','Slivers','Jeskai Ephemerate']
standard_deck_names = ['Grixis Midrange','Mono-White Midrange','5 Color Midrange','Mono-Red Aggro',
        'Esper Midrange','Mono-Blue Tempo','Azorius Soldiers','Selesnya Poison',
        'Jund Midrange','Kami War','Esper Legends','Mardu Midrange','Rakdos Aggro',
        'Domain Control','Esper Aggro']
pioneer_deck_names = ['Rakdos Midrange','Abzan Greasefang','Nykthos Ramp','Azorius Control','Temur Indomitable Creativity',
         'Lotus Field Combo','Gruul Midrange','Izzet Phoenix','5 Color Midrange','Selesnya Angels',
         'Mono-White Humans','Dimir Control','Azorius Spirits','5 Color Fires of Invention','Mono-Red Goblins',
         'Mono-Blue Spirits','Rakdos Sacrifice','4 Color Auras','Mono-Black Midrange','Bant Spirits']
modern_deck_names = ['Izzet Murktide','5 Color Indomitable Creativity','Hammer Time','Rakdos Midrange','Burn','4 Color Omnath',
         'Amulet Titan','Golgari Yawgmoth','Domain Zoo','Mono-Green Tron','Dimir Mill','Generic Ragavan','Living End',
         'Merfolk',"Death's Shadow",'Jeskai Control','5 Color Reanimator','Mono-Blue Affinity','Jeskai Prowess','Azorius Control']
legacy_deck_names = ['Reanimator','Cephalid Breakfast','Painter','Mono-Blue 8-Cast','Izzet Delver','Sneak and Show','Death\'s Shadow',
         'Five-Color Zenith','Doomsday','Death and Taxes','Temur Delver','Mono-Red Prison','Elves','Naya Depths',
         'Lands','Food Chain Goblins','Maverick','Jeskai Control','4 Color Aluren','Mono-Red Burn','Mono-White Hammer',
         'Jeskai Stoneblade','4 Color Stoneblade','Omni-Tell','Ad Nauseam Tendrils']
vintage_deck_names = ['Esper Tinker','Mono-White Initiative','Paradoxical Outcome','Doomsday','Squee Hollow Vine','Dredge','Jeskai Control',
         'Oath of Druids','Prison Shops','Aggro Shops','4 Color Tempo','Azorius Initiative','Hogaak Vine']

formats = pd.DataFrame({'format_name': format_names,
                       'format_decklists' : [pauper_deck_names,standard_deck_names,pioneer_deck_names,modern_deck_names,legacy_deck_names,vintage_deck_names]})

formats.head()


,format_name,format_decklists
0,Pauper,"[Burn, Affinity, Dimir Control, Mono-Blue Faer..."
1,Standard,"[Grixis Midrange, Mono-White Midrange, 5 Color..."
2,Pioneer,"[Rakdos Midrange, Abzan Greasefang, Nykthos Ra..."
3,Modern,"[Izzet Murktide, 5 Color Indomitable Creativit..."
4,Legacy,"[Reanimator, Cephalid Breakfast, Painter, Mono..."


In [9]:
decks = pd.DataFrame(columns=['Deck_Name','Deck_List','Format','Paper_Price','MTGO_Price_Tix'])

deck_names_column = []
deck_lists_column = []
format_column = []

for format in formats.index:
    for deck in formats['format_decklists'].loc[format]:
        deck_names_column.append(deck)
        deck_lists_column.append(pd.read_csv("Deck Lists/"+formats['format_name'].loc[format]+" Metagame Decks/"+deck+".csv"))
        format_column.append(formats['format_name'].loc[format])

decks['Deck_Name'] = deck_names_column
decks['Deck_List'] = deck_lists_column
decks['Format'] = format_column

decks.head(len(decks))

,Deck_Name,Deck_List,Format,Paper_Price,MTGO_Price_Tix
0,Burn,Quantity Card_Name 0 ...,Pauper,NaN,NaN
1,Affinity,Quantity Card_Name 0 3...,Pauper,NaN,NaN
2,Dimir Control,Quantity Card_Name 0 ...,Pauper,NaN,NaN
3,Mono-Blue Faeries,Quantity Card_Name 0 ...,Pauper,NaN,NaN
4,Orzhov Ephemerate,Quantity Card_Name 0 2...,Pauper,NaN,NaN
...,...,...,...,...,...
109,Prison Shops,Quantity Card_Name ...,Vintage,NaN,NaN
110,Aggro Shops,Quantity Card_Name 0 ...,Vintage,NaN,NaN
111,4 Color Tempo,Quantity Card_Name 0 ...,Vintage,NaN,NaN
112,Azorius Initiative,Quantity Card_Name 0 ...,Vintage,NaN,NaN


In [10]:
def paper_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in decklist.index: #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = oracle_cards.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if oracle_cards['Price_USD'].loc[price_index]:
            card_price = float(oracle_cards['Price_USD'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

def mtgo_price(decklist):
    deck_price = 0
    #with the decklist, we want to find the price from out standard dataframe, then multiply it by the quantity of card, then add that to price
    for card_index in range(len(decklist.index)): #iterate through the cards
        #get the individual card names and quantities
        card_name = decklist['Card_Name'].loc[card_index]
        card_quantity = decklist['Quantity'].loc[card_index]

        #find the card in the standard cardlist dataframe
        price_index = oracle_cards.query('Name == "'+card_name+'"')['Price_USD'].index[0]
        if oracle_cards['Price_Tix'].loc[price_index]:
            card_price = float(oracle_cards['Price_Tix'].loc[price_index])
        else:
            card_price = 0

        #add the price of the card for each time that the card 
        deck_price += (card_price * card_quantity)
        

    return deck_price

In [11]:
for i in decks.index:
    decks['Paper_Price'].loc[i] = paper_price(decks['Deck_List'].loc[i])
    decks['MTGO_Price_Tix'].loc[i] = mtgo_price(decks['Deck_List'].loc[i])

In [12]:
#finally, fix the doubled names (so tableau won't think they're the same)
#I'd like to just iterate over df.loc[df['Deck_Name'].duplicated()].index, but then we aren't hitting the first instance of the duplicate
duplicate_names = []
for i in decks['Deck_Name'].loc[decks['Deck_Name'].duplicated()].index:
    duplicate_names.append(decks['Deck_Name'].loc[i])
    #this makes duplicate_names a list of all duplicated names for reference

for i in decks.index:
    if decks['Deck_Name'].loc[i] in duplicate_names:
        decks['Deck_Name'].loc[i] = decks['Format'].loc[i]+' '+decks['Deck_Name'].loc[i]

In [13]:
decks[['Deck_Name','Format','Paper_Price','MTGO_Price_Tix']].to_csv("Format Price Data/Format_Price_Comparison.csv", index = False)